# Module 

In [78]:
# from __future__ import print_function

%load_ext autoreload
%autoreload 2

import sys 
sys.path.append("../../")
import tensorflow as tf

from src.DataLoader.mnist_data import get_mnist_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

In [79]:
debug = False

learning_rate = 0.1 
num_input = 784
num_h1 = 256
num_h2 = 256
num_label = 10

# Data 

In [80]:
mnist, total_batch, batch_size = get_mnist_data(debug=debug)
print(f"{total_batch}, {batch_size}")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
550, 100


In [81]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"image": mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, shuffle=True
)

In [82]:
mnist.train.num_examples

55000

# Model 

In [83]:
def neural_model(x):
    x = x["image"]
    x = tf.layers.dense(x, num_h1)
    x = tf.layers.dense(x, num_h2)
    x = tf.layers.dense(x, num_label)
    return x 

In [92]:

def model_fn(features, labels, mode):
    logit = neural_model(features)
    probs = tf.nn.softmax(logit)
    predict_lable = tf.argmax(probs, axis=1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        prediction = {
            "probs": probs,
            "label": predict_lable
        }
        return tf.estimator.EstimatorSpec(
            predictions=prediction,
            mode=mode
        )
#     print(labels)
#     print(logit)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=tf.cast(labels, tf.float32), logits=logit
    )
    loss = tf.reduce_mean(cross_entropy)
    optimizor = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizor.minimize(
        loss=loss,
        global_step=tf.train.get_global_step()
    )
#     print(f"label: {labels}")
#     print(f"predictions: {predict_lable}")
    accuracy = tf.metrics.accuracy(
        labels=tf.argmax(labels, axis=1),
        predictions=predict_lable
    )
#     if mode == tf.estimator.ModeKeys.TRAIN:
    prediction = {
        "prob": probs,
        "label": predict_lable
    }
    return tf.estimator.EstimatorSpec(
        predictions=prediction,
        loss=loss,
        train_op=train_op,
        eval_metric_ops={"accuracy": accuracy},
        mode=mode
    )

In [93]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp03mu670p', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcf64c90b70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Train

In [94]:
estimator.train(
    input_fn=input_fn
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp03mu670p/model.ckpt.
INFO:tensorflow:loss = 2.39172, step = 1
INFO:tensorflow:global_step/sec: 256.027
INFO:tensorflow:loss = 0.319223, step = 101 (0.392 sec)
INFO:tensorflow:global_step/sec: 257.218
INFO:tensorflow:loss = 0.534427, step = 201 (0.389 sec)
INFO:tensorflow:global_step/sec: 262.672
INFO:tensorflow:loss = 0.305554, step = 301 (0.381 sec)
INFO:tensorflow:global_step/sec: 264.538
INFO:tensorflow:loss = 0.346645, step = 401 (0.378 sec)
INFO:tensorflow:global_step/sec: 270.928
INFO:tensorflow:loss = 0.266695, step = 501 (0.369 sec)
INFO:tensorflow:Saving checkpoints for 550 into /tmp/tmp03mu670p/model.ckpt.
INFO:tensorflow:Loss for final step: 0.244408.


# Eval 

In [95]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"image": mnist.test.images}, y=mnist.test.labels, shuffle=False
)

In [96]:
estimator.evaluate(
    input_fn=test_input_fn
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-25T13:27:58Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp03mu670p/model.ckpt-550
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-04-25-13:27:59
INFO:tensorflow:Saving dict for global step 550: accuracy = 0.9076, global_step = 550, loss = 0.320767
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 550: /tmp/tmp03mu670p/model.ckpt-550


{'accuracy': 0.90759999, 'loss': 0.32076713, 'global_step': 550}

In [97]:
estimator.evaluate(
    input_fn=input_fn
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-25T13:28:26Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp03mu670p/model.ckpt-550
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-04-25-13:28:28
INFO:tensorflow:Saving dict for global step 550: accuracy = 0.908109, global_step = 550, loss = 0.32095
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 550: /tmp/tmp03mu670p/model.ckpt-550


{'accuracy': 0.90810907, 'loss': 0.32095, 'global_step': 550}

# cross entropy 

In [56]:
a = tf.constant([1, 2])
b = tf.constant(
    [
        [0, 1, 0],
        [0, 0, 1]
    ]
)
logit = tf.constant(
    [
        [0, 1, 0],
        [1, 0, 1]
    ]
)
ce_1 = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=tf.cast(logit, tf.float32),
                                                      labels=tf.cast(a, tf.int32))
ce_2 = tf.nn.softmax_cross_entropy_with_logits_v2(logits=tf.cast(logit, tf.float32),
                                                      labels=tf.cast(b, tf.float32))

In [57]:
with tf.Session() as sess:
    print(sess.run([ce_1, ce_2]))

[array([ 0.55144465,  0.8619948 ], dtype=float32), array([ 0.55144465,  0.8619948 ], dtype=float32)]
